## Extract Text from PDF

In [8]:
from pypdf import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# Example usage
pdf_text = extract_text_from_pdf("/Users/nychanthrith/Documents/Chantharith/ME-Chatbot/ai-model/data/raw_pdfs/Institute of Technology of Cambodia.pdf")
print(pdf_text)  # Print first 500 characters for checking

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 49 0 (offset 0)


Institute of Technology of Cambodia (ITC) Training Programs The Institute of Technology of Cambodia (ITC) offers a variety of training programs designed to enhance the skills and knowledge of students, professionals, and the broader community. Below is a comprehensive overview of some of the notable training initiatives provided by ITC.  1. Cambodia Energy Manager and Auditor Training Program (CEMAT) Overview: ITC collaborates with the United Nations Development Programme (UNDP) to offer the Cambodia Energy Manager and Auditor Training Program (CEMAT). This program equips participants with essential competencies in energy management and auditing. Key Highlights: • The inaugural training sessions were conducted in late 2023. • A total of 31 participants completed the first batch. • Trainees included government officials, academic professionals, and private sector representatives. • The training covers energy efficiency, auditing methodologies, and energy management best practices. For m

## Train Ollama Model on Extracted Text

In [6]:
import ollama

response = ollama.chat(model="llama2:7b", messages=[
    {"role": "system", "content": "You are an ITC chatbot trained on ITC-related documents."},
    {"role": "user", "content": f"Here is some reference material:\n{pdf_text[:2000]}\n\nNow answer: What subjects are important for ITC admission?"}
])

print(response["message"]["content"])


Based on the reference material provided, the following subjects are considered important for admission to the Institute of Technology of Cambodia (ITC):

1. Basic Programming in C/C++: This subject is crucial for understanding fundamental programming concepts and logic, which are essential skills for any IT-related field.
2. Basic Web Development: This course covers HTML, CSS, and JavaScript, which are fundamental technologies for building dynamic websites and web applications.
3. E-Learning Initiatives: As ITC's e-Learning Center is dedicated to digital education and innovation, knowledge of these subjects can provide a solid foundation for pursuing higher education in the field of information and communication technology (ICT).
4. Other subjects such as energy management, auditing methodologies, and energy management best practices are also important for those interested in pursuing training programs offered by ITC, particularly the Cambodia Energy Manager and Auditor Training Progr

In [7]:
# Start chatbot loop
print("🧠 ITC Chatbot is ready! Type 'exit' to quit.\n")

while True:
    user_query = input("You: ")  # Get user input
    
    if user_query.lower() == "exit":
        print("Chatbot: Goodbye! 👋")
        break  # Exit the loop

    # Send the user query to Ollama with PDF content as context
    response = ollama.chat(model="llama2:7b", messages=[
        {"role": "system", "content": "You are an ITC chatbot trained on ITC-related documents."},
        {"role": "user", "content": f"Here is some reference material:\n{pdf_text[:2000]}\n\nNow answer: {user_query}"}
    ])

    print("Chatbot:", response["message"]["content"])  # Print chatbot response

🧠 ITC Chatbot is ready! Type 'exit' to quit.

Chatbot: Hi there! As an ITC chatbot, I'm here to help you with any questions or information you need about Institute of Technology of Cambodia (ITC) training programs. Based on the reference material provided, here are some key points:

1. Cambodia Energy Manager and Auditor Training Program (CEMAT): This program is a collaboration between ITC and UNDP that equips participants with essential competencies in energy management and auditing. The training covers energy efficiency, auditing methodologies, and energy management best practices.
2. GIC Summer Training Program: The Department of Information and Communication Technology (GIC) at ITC organizes an annual summer training program to enhance technical skills among students and professionals. The courses cover basic programming in C/C++, basic web development, and other topics.
3. E-Learning Initiatives: ITC's e-Learning Center is dedicated to digital education and innovation, and the ins

In [9]:
import ollama

response = ollama.chat(model="custom_itc_bot", messages=[
    {"role": "user", "content": "What is the Cambodia Energy Manager and Auditor Training Program?"}
])

print(response["message"]["content"])



Greetings! As an ITC chatbot, I'm glad to assist you with any questions or queries related to ITC. The Cambodia Energy Manager and Auditor Training Program is a comprehensive training initiative designed for energy managers and auditors in Cambodia. The program aims to enhance the knowledge and skills of these professionals in the field of energy management and auditing, with a focus on sustainable energy practices and energy efficiency.

The program covers various aspects of energy management and auditing, including energy conservation techniques, energy-efficient technologies, energy audits, and energy reporting. It also provides hands-on training on energy-related software tools and equipment. The program is designed to improve the capacities of energy managers and auditors in Cambodia to manage energy efficiently and effectively, and to promote sustainable energy practices in the country.

The program consists of several modules, including:

1. Introduction to Energy Management an

In [13]:
# Define function to query the model
def query_chatbot(query: str):
    response = ollama.chat(model="custom_itc_bot", messages=[
        {"role": "user", "content": query}
    ])
    return response["message"]["content"]

# Example query
query = "What is institute of technology of cambodia?"
print(query_chatbot(query))

Thank you for asking! The Institute of Technology of Cambodia (ITC) is a leading institution of higher education in Cambodia, established in 1965. ITC offers undergraduate and graduate programs in a variety of fields, including engineering, computer science, business administration, and more.

ITC has a strong reputation for providing high-quality education, with a focus on practical training and hands-on experience. The institute has a diverse student body and faculty, with a commitment to promoting social responsibility and community engagement. ITC also has partnerships with several international universities and organizations, providing students with opportunities for exchange programs and research collaborations.

In addition to its academic programs, ITC is also involved in various research projects and community outreach initiatives, aimed at addressing some of the social and economic challenges facing Cambodia. These include projects related to renewable energy, water resource 